# Numpy and numba experiment

This follows [this linkedin post](https://www.linkedin.com/posts/habib-boukrana-755479175_python-numba-optimisation-activity-7244822650022244352-3_Q0?utm_source=share&utm_medium=member_desktop) and aims to compare numba njit to optimization using numpy.

In [5]:
import numpy as np
from numba import njit
import timeit

In [6]:
def func_without_numba():
    result = []
    for a in range(10000):
        for b in range(10000):
            if (a+b)%11 == 0:
                result.append((a,b))
    return result

@njit
def func_with_numba():
    result = []
    for a in range(10000):
        for b in range(10000):
            if (a+b)%11 == 0:
                result.append((a,b))
    return result

def func_numpy_v1():
    a, b = np.meshgrid(np.arange(10000), np.arange(10000))
    mask = (a + b) % 11 == 0
    return np.column_stack((a[mask], b[mask]))

def func_numpy_v2():
    a = np.arange(10000)
    b = np.arange(10000)
    mask = ((a[:, np.newaxis] + b) % 11) == 0
    return np.argwhere(mask)

def func_numpy_v3():
    n = 10000
    a = np.arange(n)[:, np.newaxis]
    
    # Calculate the smallest 'b' for each 'a' that satisfies (a + b) % 11 == 0
    b = 11 - (a % 11)
    b %= 11  # Ensure b is always in the range [0, 10]
    
    # Generate all possible 'b' values by adding multiples of 11
    multiples_of_11 = 11 * np.arange(n // 11 + 1)
    b = b + multiples_of_11
    
    # Keep only valid pairs where b < 10000
    mask = b < n
    return np.column_stack((np.broadcast_to(a, b.shape)[mask], b[mask]))

def func_numpy_v4():
    n = 10000
    a, b = np.ogrid[:n, :n]
    mask = (a + b) % 11 == 0
    return np.argwhere(mask)

In [7]:
# Timing function
def time_function(func, name):
    start = timeit.default_timer()
    result = func()
    end = timeit.default_timer()
    print(f"{name:<20} Time: {end-start:.4f} s, Size: {len(result)}")
    return result, end-start

In [8]:
# Run and time all functions
print("Running and timing functions...")
result_orig, time_orig = time_function(func_without_numba, "Without Numba")
result_numba, time_numba = time_function(func_with_numba, "With Numba")
result_numpy_v1, time_numpy_v1 = time_function(func_numpy_v1, "NumPy v1")
result_numpy_v2, time_numpy_v2 = time_function(func_numpy_v2, "NumPy v2")
result_numpy_v3, time_numpy_v3 = time_function(func_numpy_v3, "NumPy v3")
result_numpy_v4, time_numpy_v4 = time_function(func_numpy_v4, "NumPy v4")

Running and timing functions...
Without Numba        Time: 4.2702 s, Size: 9090910
With Numba           Time: 0.9970 s, Size: 9090910
NumPy v1             Time: 1.2485 s, Size: 9090910
NumPy v2             Time: 0.8243 s, Size: 9090910
NumPy v3             Time: 0.0900 s, Size: 9090910
NumPy v4             Time: 0.7364 s, Size: 9090910


In [9]:
# Verify results
print("\nVerifying results...")
orig_set = set(map(tuple, result_orig))
def verify(result, name):
    result_set = set(map(tuple, result))
    if result_set == orig_set:
        print(f"{name} results match.")
    else:
        print(f"{name} results differ.")
        print(f"  Missing pairs: {len(orig_set - result_set)}")
        print(f"  Extra pairs: {len(result_set - orig_set)}")
        if len(orig_set - result_set) > 0:
            print(f"  Sample missing: {list(orig_set - result_set)[:5]}")
        if len(result_set - orig_set) > 0:
            print(f"  Sample extra: {list(result_set - orig_set)[:5]}")

verify(result_numpy_v1, "NumPy v1")
verify(result_numpy_v2, "NumPy v2")
verify(result_numpy_v3, "NumPy v3")
verify(result_numpy_v4, "NumPy v4")
verify(result_numba, "Numba")


Verifying results...
NumPy v1 results match.
NumPy v2 results match.
NumPy v3 results match.
NumPy v4 results match.
Numba results match.


In [10]:
# Calculate and print speedups
print("\nPerformance comparison:")
print(f"{'Method':<15}{'Time (s)':<12}{'Speedup':<10}")
print("-" * 37)
print(f"{'Without Numba':<15}{time_orig:.4f}{' ':12}{'1.00x':<10}")
print(f"{'With Numba':<15}{time_numba:.4f}{' ':12}{time_orig/time_numba:.2f}x")
print(f"{'NumPy v1':<15}{time_numpy_v1:.4f}{' ':12}{time_orig/time_numpy_v1:.2f}x")
print(f"{'NumPy v2':<15}{time_numpy_v2:.4f}{' ':12}{time_orig/time_numpy_v2:.2f}x")
print(f"{'NumPy v3':<15}{time_numpy_v3:.4f}{' ':12}{time_orig/time_numpy_v3:.2f}x")
print(f"{'NumPy v4':<15}{time_numpy_v4:.4f}{' ':12}{time_orig/time_numpy_v4:.2f}x")


Performance comparison:
Method         Time (s)    Speedup   
-------------------------------------
Without Numba  4.2702            1.00x     
With Numba     0.9970            4.28x
NumPy v1       1.2485            3.42x
NumPy v2       0.8243            5.18x
NumPy v3       0.0900            47.44x
NumPy v4       0.7364            5.80x
